In [1]:
import docx
import os
import re
from CurriculumDB.Modelsn4j import *
from neo4j import GraphDatabase


In [13]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = ("curriculum", "mycurriculum")

driver = GraphDatabase.driver(URI, auth=AUTH)
print(driver)
print(driver.verify_connectivity())
#help(driver.verify_connectivity)
factory = CurriculumFactory(driver, 'curriculumdb')

None


In [ ]:
help(factory.get_or_create_Element)

In [3]:
moduledir = "C:/Users/dmamartin/OneDrive - University of Dundee/Teaching/CurriculumDB/Modules/"

In [4]:
modules = {}
for stage in os.listdir(moduledir):
    print(f'Looking in {stage}')
    for file in os.listdir(os.path.join(moduledir, stage)):
        print(f'Checking {file}')
        if file.endswith('.docx'):
            print(f'processing {file}')
            doc=docx.Document(os.path.join(moduledir, stage, file))
            structure ={}
            for t in doc.tables:
                for r in range(len(t.rows)):
                    structure[t.row_cells(r)[0].text.split()[0]] =t.row_cells(r)[1].text
            modules[structure['1.2']]=structure

Looking in Stage 1
Checking ARCHIVE
Checking BS10001 Widening Access Programme Collective Module Specification QASv1.docx
processing BS10001 Widening Access Programme Collective Module Specification QASv1.docx
Checking BS11005 Numeracy, Chemistry and Physics for the Biological and Biomedical Sciences Spec QASv4.docx
processing BS11005 Numeracy, Chemistry and Physics for the Biological and Biomedical Sciences Spec QASv4.docx
Checking BS11006 The Poison Pen Specification QASv4.docx
processing BS11006 The Poison Pen Specification QASv4.docx
Checking BS11008 Core Skills in the Life Sciences 1A QASv3.docx
processing BS11008 Core Skills in the Life Sciences 1A QASv3.docx
Checking BS11009 The Building Blocks of Life QASv2.docx
processing BS11009 The Building Blocks of Life QASv2.docx
Checking BS12005 Science and Society Module Spec QASv4.docx
processing BS12005 Science and Society Module Spec QASv4.docx
Checking BS12008 Introduction to Scientific Enterprise Specification QASv4.docx
processing

Checking BS32707 Molecular Microbiology in Human Disease Module Specification QASv1.docx
processing BS32707 Molecular Microbiology in Human Disease Module Specification QASv1.docx
Checking BS32708 Molecular Biophysics Module Specification QASv1.docx
processing BS32708 Molecular Biophysics Module Specification QASv1.docx
Looking in Stage 4
Checking ARCHIVE
Checking BS41004 Research Project Biological Sciences Module Specification QASv4.docx
processing BS41004 Research Project Biological Sciences Module Specification QASv4.docx
Checking BS41005 Research Project Biological Sciences Programme Specification QASv3.docx
processing BS41005 Research Project Biological Sciences Programme Specification QASv3.docx
Checking BS41006 Research Skill in Biomedical Sciences QASv5.docx
processing BS41006 Research Skill in Biomedical Sciences QASv5.docx
Checking BS41007 Research Project Biomedical Sciences Module Specification QASv6.docx
processing BS41007 Research Project Biomedical Sciences Module Speci

In [5]:
print(Module.requiredParams, Module.optionalParams)


{'code': 'Module code', 'name': 'Module name', 'credits': 'Credit weight', 'scqflevel': 'SCQF level', 'shelevel': 'SHE level', 'semester': '1, 2 or both'} {'block': 'Teaching block: C or D', 'startyear': 'First academic year', 'endyear': 'Last academic year'}


In [152]:
newmod = modules['BS32031']
modobj ={}

scqf = re.compile(r'SCQF\D+(\d+)\D+(\d+)\D')
delivery = re.compile(r'Semester (\d)')
#for m in modules:
#    print(m)
#for m in ['BS32031']:
for m in modules:
    try:
        newmod=modules[m]
    
        match=scqf.search(newmod['1.3'])
        scqflevel=0
        shelevel=0
        credits=0
        if match:
            scqflevel = int(match.group(1))
            shelevel = scqflevel - 6
            credits = int(match.group(2))
        else: 
            shelevel = int(m[2])
            scqflevel = shelevel +6
            print(m,"1.3", newmod['1.3'])
        match=delivery.search(newmod['1.14'])
        semester=0
        if match:
            semester = int(match.group(1))
        else:
            print(m,"1.14",newmod['1.14'])
            semester = int(m[3])
        if credits and scqflevel and shelevel and semester:
            newmodparams ={'code':newmod['1.2'], 'name':newmod['1.1'], 'semester':semester, 
                           'credits': credits, 'scqflevel':scqflevel, 'shelevel':shelevel}
            modobj[m]=factory.get_or_create_Element('Module', **newmodparams)
    except Exception as e:
        print(newmodparams,e)
        

BS10001 1.14 
BS11008 1.14 
BS11009 1.14 
BS12009 1.14 
BS12010 1.14 
BS21002 1.14 
BS21012 1.14 
BS30007 1.14 
BS32034 1.14 
BS41009 1.14 
BS42028 1.14 
BS51007 1.14 
BS51008 1.14 
BS51009 1.14 
BS51010 1.14 
BS51011 1.14 
BS51012 1.14  
BS52007 1.14 
BS52009 1.14 
BS52010 1.14 
BS52011 1.14 
BS52012 1.14 
BS52013 1.14 Once per academic year in semester 2
BS53003 1.14 
BS53004 1.14 
BS53005 1.14  semester 3 (summer) 
BS53006 1.14  semester 3 (summer) 
BS53007 1.14 


In [162]:
for m in modules:
    if modules[m]['1.16'].strip() and modules[m]['1.16'].strip().upper()[0]!='N':
        print(f'{m}: Prerequisites: {modules[m]["1.16"]}')
        modules[m]['Pre']=modules[m]["1.16"]
    if modules[m]['1.17'].strip() and modules[m]['1.17'].strip().upper()[0]!='N':
        print(f'{m}: Corequisites: {modules[m]["1.17"]}')
        modules[m]['Co']=modules[m]["1.17"]
    if modules[m]['1.18'].strip() and modules[m]['1.18'].strip().upper()[0]!='N':
        print(f'{m}: Antirequisites: {modules[m]["1.18"]}')
        modules[m]['Anti']=modules[m]["1.18"]
    

BS11005: Prerequisites: After completion of the skills audit prior to matriculation, a decision on whether this module is to be taken by the student will be made
BS11005: Corequisites: For students enrolled for Biological and Biomedical Sciences degrees, the following are all co-requisites:
   BS11009 (knowledge based)
   BS11008 (Lab and skills-based)

BS11008: Corequisites: For students enrolled for Biological Sciences and Biomedical Sciences degrees, the following module is a co-requisites:
• BS11009 (Knowledge based)
BS12010: Corequisites: For students enrolled for Biological Sciences and Biomedical Sciences degrees, the following modules are all co-requisites:
• BS12001 (Knowledge based)
• BS12002 (Knowledge based)
BS12701: Prerequisites: Successful completion of JPD Core curriculum to date
BS12702: Prerequisites: Successful completion of JPD Core curriculum to date
BS21001: Prerequisites: BS11001, BS11002, BS11008, BS12001, BS12002, BS12010 or Level 2 direct entry requirements

B

In [158]:
for t in doc.tables:
    for r in range(len(t.rows)):
                    print(t.row_cells(r)[0].text)

Applicability
Section 1: Module specification
Heading
1.1 Module title
1.2 Module code
1.3 Scottish Credit and Qualifications Framework (SCQF) level and credits
1.4 Aims
1.5 Indicative content
1.6 Intended learning outcomes (ILOs)
1.7 Assessment strategy
1.8 Assessment types and weighting
1.9 Teaching and learning
1.10 Indicative student contact hours and the expected amount of independent study
1.11 Teaching staff
1.12 Mode of study
1.13 Location(s) for the delivery of teaching
1.14 When delivered 
1.15 Equality and diversity
1.16 Pre-requisites or entry requirements
1.17 Co-requisites
1.18 Anti-requisites
1.19 Student numbers
Section 2: Responsibilities and contacts
Heading
2.1 Lead School
2.2 Lead discipline
2.3 Collaborating Schools (where relevant)
2.4 Module leader
2.5 Lead administrator for the module
Section 3: Module details (information for internal use for quality assurance and management)
Heading
3.1 Rationale for the module
3.2 First student intake
3.3 External organisatio

In [128]:

class Node():
    '''Represents a Node in a Neo4j database. 
    Contains methods for accessing links, ensuring that on creation the 
    correct parameters are given etc.'''

    ntype='Node'
    
    requiredParams = {
        }
    optionalParams= {}
    
    def __init__(self, factory=None,  **kwparams):
        '''Initialiser. Checks required parameters are present'''
        if factory is None:
            raise Exception('Curriculum factory must be given as factory')
        self.factory = factory
        self.ntype=self.__class__.__name__.split('.')[-1]
        self.paramChecks={'default': self._checkDefault}
        self.params={}
        self.edges=[]
        self._get_labels()
        for par in self.requiredParams:
            if not kwparams.get(par):
                print(par, kwparams)
                raise InsufficientParameterSpecException(f'Missing parameter <{par}> of type <{self.requiredParams[par]}>')
            if not self.paramChecks.get(par,self._checkDefault)(kwparams[par]):
                raise BadParameterException(f'Parameter <{par}> should be of type <{self.requiredParams[par]}>')
            self.params[par]=kwparams[par]
        for par in self.optionalParams:
            if par not in kwparams:
                continue
            if not self.paramChecks.get(par,self._checkDefault)(kwparams[par]):
                raise BadParameterException(f'Parameter <{par}> should be of type <{self.requiredParams[par]}>')
            self.params[par]=kwparams[par]
        if 'elementID' not in kwparams:
            self._create_node()
        else:
            self.element_id=kwparams['elementID']
            self.edges =self.getEdges()
    def __str__(self):
        '''Returns a string represntation of the node'''
        params = ", ".join([f'{k}: {self.params[k]}' for k in self.requiredParams])
        return f'{self.ntype}: [{params}] ID: {self.element_id}'
    
    def __repr__(self):
        return self.__str__()
                            
    def _create_node(self):
        '''Internal method that creates a new instance of the node, if it does not exist'''
        paramlist = ", ".join([f'{k}: ${k}' for k in self.params])
        paramlist = "{"+paramlist+"}"
        nodelabels = ':'.join(self.labels)
        query= f"MERGE (a:{nodelabels} {paramlist}) RETURN a"
        records,summary, keys = self.factory.db.execute_query(query, self.params, database_=self.factory.dbname)   
        self.element_id = records[0].items()[0][1].element_id
            
    def _checkDefault(self, value):
        '''default placeholder for parameter value checks. Returns True'''
        return True
    
    def _checkAY(self, value):
        '''Checks for a correctly formatted Academic year as:
            2425
            24/25
            2024/5'''
        #TODO 
        return True
    def getparam(self, prop, default=None):
        '''
        Returns a parameter value (default if not found)

        Parameters
        ----------
        prop : property name
            
        default : TYPE, optional
             The default is None.

        Returns
        -------
        Property value
            
        '''
        return self.params.get(prop,default)
    
    def setparam(self, key, value):
        '''
        Sets the parameter key to value. No type checking.

        Parameters
        ----------
        key : legal value for a dictionary key
            
        value : Value to store.
        
        Returns
        -------
        None.

        '''
        self.params[key]=value
        self.update()
    
    def getEdges(self,relation=None,**kwargs):
        '''retrieves edges connected to or from the Node.
        Relation is the type of relation to retrieve (default is all)
        other arguments are applied as matching parameters'''
        edges=[]
        filterstring=''
        filterparams={}
        for kw in kwargs:
            if kw != 'id':
                filterstring += f"'{kw}': ${kw},"
                filterparams[kw] = kwargs[kw]
        rel=''
        if relation:
            rel=f":{relation}"
        filterparams['id'] = self.element_id
        if filterstring:
            filterstring = "{"+filterstring+"}"
        records, summary, keys = self.factory.db.execute_query(
        f"MATCH (p: {self.ntype} {{id: $id }}) -[e{rel} {filterstring}]-(q) RETURN e,q",
        filterparams,
        routing_=neo4j.RoutingControl.READ,  # or just "r"
        database_=self.factory.dbname)
        for edge in records:
            relation = edge.items[0][1]
            target = edge.items[1][1]
            edges.append({'source': self,'edge': relation, 'target': target})
        return edges
    def isCurrent(self, year):
        '''
        If the edge has a start/end year, check if the given year is the period in which the relation is valid

        Returns
        -------
        Boolean: True if the eyar is between start and end (inclusive), after start if no end, or True if no start

        '''
        
        if self.params.get('startyear'):
            if AcademicYear(year)< AcademicYear(self.params.get('startyear')):
                return False
            if self.params.get('endyear'):
                if AcademicYear(year)> AcademicYear(self.params.get('endyear')):
                    return False
        return True
    
    
    def toDict(self):
        retp ={'element_id': self.element_id}
        for p in self.params:
            retp[p]=self.params[p]
        return retp
    
    def update(self, **kwargs):
        '''
        Update named parameters for a Node

        Parameters
        ----------
        **kwargs : key,value parameter set
            any key,value parameters. Existing values will be overwritten.
            No type checking

        Returns
        -------
        None.

        '''
        paramlist =[]
        if not kwargs:
            return
        for k in kwargs:
            if k !='id':
                paramlist.append( f'a.{k}= ${k}')
                self.params[k]=kwargs[k]
        kwargs['id'] = self.element_id    
        paramlist = "{"+paramlist+"}"
        query= f"MERGE (a:{self.ntype}) WHERE elementID(a) = $id SET {', '.join(paramlist)}"
        records,summary, keys = self.factory.db.execute_query(query,kwargs, database_=self.factory.dbname)   
        
    def _get_labels(self):
        self.labels = [x.__name__ for x in globals()[self.__class__.__name__].__mro__]
        self.labels.remove('object')
        self.labels.remove('Node')
        
    
    
class Edge():
    '''Represents an edge in a Neo4J database'''
    requiredParameters={
        'relation': 'relationship type',
        'source': 'Node ',
        'target': 'Node '}
    optionalParams={
        'startyear': 'Academic year start',
        'endyear': 'Academic year to'
        }
    def __init__(self, factory, **kwargs):
        '''Edge between two nodes. Must specify source, target and relationship type.'''
        for kw in self.requiredParameters:
            if kw not in kwargs:
                raise InsufficientParameterSpecException(f"{kw} not specified in Edge constructor")
            self.params[kw]=kwargs[kw]
    def set_expiry(self, expirydate):
        '''
        Sets the 'to' parameter with an academic year for when this relation was withdrawn. 
        The date specified will be the last academic year for which it was valid.

        Parameters
        ----------
        expirydate : Academic year
            Academic year is an academic year object or text description of it.

        Returns
        -------
        None.

        '''
        self.params['to'] =AcademicYear(expirydate)
    
    def isCurrent(self, year):
        '''
        If the edge has a start/end year, check if the given year is the period in which the relation is valid

        Returns
        -------
        Boolean: True if the eyar is between start and end (inclusive), after start if no end, or True if no start

        '''
        
        if self.params.get('startyear'):
            if AcademicYear(year)< AcademicYear(self.params.get('startyear')):
                return False
            if self.params.get('endyear'):
                if AcademicYear(year)> AcademicYear(self.params.get('endyear')):
                    return False
        return True
    def getSource(self):
        '''
        Returns a Python object representing the source node

        Returns
        -------
        None.

        '''
        
class CurriculumFactory():
    
    def __init__(self, db, dbname='curriculum'):
        self.db = db
        self.dbname = dbname
        self.programmecache = {}
        self.modulecache = {}
        self.activitycache = {}
        self.personcache = {}
        
    def get_connection(self):
        return self.db
    
    @lru_cache    
    def get_element_by_ID(self, elementID):
        '''
        Retrieves an element by ID and creates the appropriate node for it, if it is a node.
        
        Parameters
        ----------
        elementID : Internal neo4j element ID
            text string uniquely identifying the object in the database.

        Returns
        -------
        Object of correct class, if available or None if not found.

        '''
        erecords, _, _ = self.db.execute_query(
            "MATCH (a ) where elementId(a)=$id"
            "RETURN a", id=elementID,
             database_=self.dbname,
        )
        if not erecords :
            return
        nodeclass = list(erecords[0].items()[0][1].labels())[0]
        params = dict(erecords[0].items()[0][1])
        params['elementID']=elementID
        if nodeclass in globals():
            return globals()[nodeclass](**params)
        
      
    def get_or_create_Element(self, ElementName,**kwargs):
        '''
        Generic get and create method. This requires all elements to have a unique id under the property code

        Parameters
        ----------
        ElementName : Name of element to create, eg Programme
            This will directly map to the class name.
        **kwargs : Parameters to use. These will be checked against every class.
            

        Returns
        -------
        Element of type ElementName on success

        '''
        
        elem= None
        
        if ElementName not in globals():
            return
        ElementClass=globals()[ElementName]
        if kwargs.get('elementID',None):
            elem = self.get_element_by_id(kwargs['elementID'])
            return elem
        if kwargs.get('code'):
            try:
                result,_,_=self.db.execute_query(f'MATCH (p:{ElementName} {{code: $name}} ) return p', name=kwargs['code'], database_=self.dbname)
                if result:
                    params=dict(result[0].items()[0][1])
                    params['elementID']=result[0]['p'].element_id
                    elem = ElementClass(self, **params)
                    return elem
            except Exception as e:
                e.add_note(str(params))
                raise e
                
        for p in ElementClass.requiredParams:
            if p not in kwargs:
                raise InsufficientParameterSpecException(f'Not enough parameters specified for {ElementName}: missing{p}')
        
        elem = ElementClass(self, **kwargs)
        return elem


    def get_all_elements(self, ElementName, **params):
        '''
        Returns all elements of type ElementName

        Parameters
        ----------
        ElementName : name of an element
            This can be any element type for which a model exists.
        **params : Keyword parameters to use in the search
            Exact matches only at present.

        Returns
        -------
        List of elements

        '''
        
        elements = []
        if ElementName not in globals():
            return elements
        paramdetails =[]
        paramtext=''
        for p in params:
            paramdetails.append(f'{p}: ${p}')
        if paramdetails:
            paramtext= f'{ {",".join(paramdetails)} }'
        erecords, _, _ = self.db.execute_query(
            f"MATCH (a :{ElementName} {paramtext})",
            params,
             database_=self.dbname,
        )
        if not erecords :
            return elements
        for item in erecords:
            elem=item.items()[0][1]
            nodeclass = list(elem.labels())[0]
            params = dict(elem)
            params['elementID']=item.element_id
            if nodeclass in globals():
                elements.append( globals()[nodeclass](self, **params))
        return elements  
    def getElementsForElement(self, element, target, max_steps=2,relation=None):
        '''
        Retreive all elements of type Target linked to element,optionally by relation (or all relations) 
        at a maximum distance of max_steps (default 2)
        
        Parameters
        ----------
        element : TYPE
            DESCRIPTION.
        target : TYPE
            DESCRIPTION.
        max_steps: integer
            Maximum link number to explore (default 2)
        relation: text
            Limit relations to those of type relation
        Returns
        -------
        List of Node type objects.
        '''
        if target not in globals():
            return []
        cypher = f"MATCH (e) --{{1,{max_steps}}} (thing:$target) where elementID(e)=$id RETURN DISTINCT thing"
        result,_,_=self.db.execute_query(cypher, target=target, id=element.element_id,database_=self.dbname)
        elems = []
        if result:
            for elem in result:
                item = elem.items()[0][1]
                params =dict(item)
                params['elementID'] = item.element_id
                elems.append(globals()[target](self, **params))
        return elems
         
class Programme(Node):
    
    DRAFT = 0
    CURRENT = 1
    ARCHIVED = 2
    WITHDRAWN =3
    ntype='Programme'
    requiredParams = {
        'code': 'Programme code',
        'name': 'Programme name'
        }
    optionalParams= {
        'startyear':'First academic year',
        'endyear': 'Last academic year',
        'school':'School which manages the Programme'}
    def __init__(self, factory, **params ):
        '''
        Create a Programme instance.

        Parameters
        ----------
        factory - database connection and cache for Programme
        **params : 
            Create or replace table Programme (
        '''
        super().__init__(factory,**params)
        
        self.modules = {}
        self.ILO = []
        if self.element_id:
            self.loadmodules()
            self.loadILO()
        

    
    
    
    def loadmodules(self):
        '''
        Loads module links from the database to the modules list. Does not create now entries in the database.

        Returns
        -------
        None.

        '''
        self.modules={}
        cypher = "MATCH (p:Programme ) -[a]-(b:Module) WHERE elementID(p) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation = t.items()[0][1]
            target = t.items()[1][1]
            if target.data()['code'] not in self.modules:
                self.modules[target.data()['code']]=[]
            self.modules[target.data()['code']].append({'relation': {'type': relation.type, 'params':dict(relation)}, 
                                                        'target':{ 'moduleID': target.element_id,'params':dict(target)}})
            
        

        
    def loadILO(self):
        '''
        Loads ILO links from the database. Does not create new links or ILOs.

        Returns
        -------
        None.

        '''
        self.ILO = {}
        cypher = "MATCH (p:Programme ) -[a]-(b:ProgrammeILO) WHERE elementID(p) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation, target = t.items()[0:2]
            
            self.ILO[target.element_id]= (dict(target), dict(relation))
        
 
        
    def withdraw(self, year):
        '''
        Set the Programme to WITHDRAWN. Only possible if there are no future versions.

        Returns
        -------
        None.

        '''
        if AcademicYear(year) and AcademicYear(year) > AcademicYear(self.params['startyear']):
            self.params['endyear']=year
            cypher = 'MATCH (p:Programme) where elementID(p)=$id SET p.endyear=$year'
            records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id,year=year, database_=self.factory.dbname)
        

    def map_module(self, module, optional=0, year=None, remove=False):
        '''
        Add a module to the Programme, updating if exisitng  already appended.

        Parameters
        ----------
        module : Module object
            DESCRIPTION.
        optional: Boolean
        remove: Academic Year for last instance
        Returns
        -------
        None.

        '''
        relations=('IS_CORE','IS_ELECTIVE')
        relation= relations[int(bool(optional))]
        cypher1 = 'MATCH (p:Programme) -[b:{relation}]-(m:Module) where elementID(p)=$pid and elementID(m)=$mid SET b.endyear=$year'
        
        if module.params['code'] in self.modules:
            if remove:
                
                records,_,_ =self.factory.db.execute_query(cypher1.format(relation=relation), 
                                                           pid=self.element_id,year=year, 
                                                           mid=module.element_id,
                                                           database_=self.factory.dbname)
                for r in self.modules[module.params['code']]:
                    if r['relation']['type']==relation:
                        r['relation']['params']['endyear']=year
            else:
                for r in self.modules[module.params['code']]:
                    if r['relation']['type']!=relation:
                        r['relation']['params']['endyear']=year
                        records,_,_ =self.factory.db.execute_query(cypher1.format(relation=relations[(int(bool(optional))+1)%2]), 
                                                                   pid=self.element_id,year=year, 
                                                                   mid=module.element_id,
                                                                   database_=self.factory.dbname)
         
        if not remove:
            if module.params['code'] not in self.modules:
                 self.modules[module.params['code']]=[]
            cypher2 =f'MERGE (p:Programme) <-[{relation} {{startyear:$year }}] - (m:Module) where elementID(p)=$pid and elementID(m)=$mid'
            records,_,_ =self.factory.db.execute_query(cypher2, 
                                                        pid=self.element_id,year=year, 
                                                        mid=module.element_id,
                                                        database_=self.factory.dbname)
        self.loadmodules()
           
                                           
               
           
           
    def map_ilo(self, ilo, year, remove=False):
        '''
        Associates or updates a Programme ILO mapping to a Programme.

        Parameters
        ----------
        ilo : ProgrammeILO 
            Programme ILO object
        year : text describing the academic year
            Must be either the start year, or for removing an ILO the last year for which it will be relevant.
        remove : Boolean, optional
            Flag to say whether the ILO should be dissociated from teh Programme. The default is False.

        Returns
        -------
        None.

        '''
        
        if not AcademicYear(year):
            raise UnparseableYearException(f'Cannot parse year value {year}')
        if ilo.element_id in self.ILO:
            if remove:
                self.ILO[ilo.element_id]['endyear']=year
                cypher="MATCH (p: Programme) -[b]- (i:ProgrammeILO) WHERE elementID(p) =$pid AND elementID(i) =$iid SET b.endyear=$year"
                records,_,_ = self.factory.db.execute_query(cypher,pid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
            return
        if remove:
            return
        cypher = "MERGE (p:Programme) -[b:HAS_ILO {startyear:$year}]->(i:ProgrammeILO) where elementID(p)=$pid AND elementID(i) = $iid RETURN b"
        records,_,_ = self.factory.db.execute_query(cypher,pid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
        if records:
            relation=records[0].items()[0][1]
            self.ILO[ilo.element_id]=(dict(ilo),dict(relation))


    
class Module(Node):
    
    DRAFT = 0
    CURRENT = 1
    ARCHIVED = 2
    WITHDRAWN = 3
    PREREQUISITE = 1
    COREQUISITE = 0
    ANTIREQUISITE = 2
    ntype='Module'
    requiredParams = {
        'code': 'Module code',
        'name': 'Module name',
        'credits': 'Credit weight',
        'scqflevel': 'SCQF level',
        'shelevel': 'SHE level',
        'semester': '1, 2 or both'
        }
    optionalParams= {
        'block': 'Teaching block: C or D',
        'startyear':'First academic year',
        'endyear': 'Last academic year'
        }
    
    def __init__(self, factory,  **params ):
        '''
        Create a Programme instance.

        Parameters
        ----------
        factory - database connection and cache for Module
        **params : 
                    '''
        super().__init__(factory,**params)
        
        self.ILO={}
        self.Activities={}
        self.Assessments = {}
        if self.element_id:
            self.loadILO()
            self.loadConstraints()
            self.loadActivities()
            self.loadAssessments()
            
    def loadActivities(self):
        '''
        Loads module links from the database to the modules list. Does not create now entries in the database.

        Returns
        -------
        None.

        '''
        self.Activities=[]
        cypher = "MATCH (m:Module ) -[a]-(b:TeachingActivity) WHERE elementID(m) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation = t.items()[0][1]
            target = t.items()[1][1]
            self.Activities.append({'relation': {'type': relation.type, 'params':dict(relation)}, 
                                                        'target':{ 'TeachingActivityID': target.element_id,'params':dict(target)}})
            
        
        
        
    def loadILO(self):
        '''
        Loads ILO links from the database. Does not create new links or ILOs.

        Returns
        -------
        None.

        '''
        self.ILO = {}
        cypher = "MATCH (m:Module ) -[a]-(b:ModuleILO) WHERE elementID(m) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation, target = t.items()[0:2]
            self.ILO[target.element_id]= (dict(target),dict(relation))
        

    def loadConstraints(self):
        '''
        Loads in but does not create constraint

        Returns
        -------
        None.

        ''' 
        self.requisites = {}
        cypher = "MATCH (m:Module ) <-[a]- (b:Module) WHERE elementID(m) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation, target = t.items()[0:2]
            relationdata = dict(relation)
            relationdata['element_id']=relation.element_id
            relationdata['type']=relation.type
            targetdata = dict(target)
            targetdata['element_id']=target.element_id
            self.requisites.append({'constraint':relationdata, 'module': targetdata})
    def loadAssessments(self):
        '''Retrieve any linked assessments from the database
        # TODO
        '''
        #TODO
        
    def setConstraint(self, constraint, module, year, remove=False):
        '''
        Constraint is an integer corresponding to the constraint type. 
        module is teh module acting as a constraint.
        remove is boolean actig as to whether a constraint is added/updated (false) or removed (true)

        Parameters
        ----------
        constraintlist : TYPE
            DESCRIPTION.

        Returns
        -------
        None.

        '''
        #TODO
            
        
        
        
    def map_ilo(self, ilo, year, remove=False):
        '''
        Associates or updates a Module ILO mapping to a Module.
    
        Parameters
        ----------
        ilo : ModuleILO 
            Module ILO object
        year : text describing the academic year
            Must be either the start year, or for removing an ILO the last year for which it will be relevant.
        remove : Boolean, optional
            Flag to say whether the ILO should be dissociated from the Module. The default is False.
    
        Returns
        -------
        None.
    
        '''
        
        if not AcademicYear(year):
            raise UnparseableYearException(f'Cannot parse year value {year}')
        if ilo.element_id in self.ILO:
            if remove:
                self.ILO[ilo.element_id]['endyear']=year
                cypher="MATCH (m: Module) -[b]- (i:ModuleILO) WHERE elementID(m) =$mid AND elementID(i) =$iid SET b.endyear=$year"
                records,_,_ = self.factory.db.execute_query(cypher,mid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
            return
        if remove:
            return
        cypher = "MERGE (m:Module) -[b:HAS_ILO {startyear:$year}]->(i:ModuleILO) where elementID(m)=$mid AND elementID(i) = $iid RETURN b"
        records,_,_ = self.factory.db.execute_query(cypher,mid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
        if records:
            relation=records[0].items()[0][1]
            self.ILO[ilo.element_id]=(dict(ilo),dict(relation))
    
    def map_activity( self, activity, relationtype='HAS_ACTIVITY'):
        '''
        Adds an activity to a module

        Parameters
        ----------
        activity : TeachingActivity

        Returns
        -------
        None.

        '''
        if activity.element_id not in [x['target']['TeachingActivityID'] for x in self.Activities]:
            cypher = f"MERGE (m:Module ) -[a:{relationtype}]-(b:TeachingActivity) WHERE elementID(m) = $id AND elementID(b)=$tid RETURN a,b"
            records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, tid=activity.element_id, database_=self.factory.dbname)
            for t in records:
                relation = t.items()[0][1]
                target = t.items()[1][1]
                self.Activities.append({'relation': {'type': relation.type, 'params':dict(relation)}, 
                                                            'target':{ 'TeachingActivityID': target.element_id,'params':dict(target)}})
        

        
# class TeachingActivityType(Node): 
    
#     ntype='TeachingActivityType'
#     requiredParams = {
#         'code': 'Module code',
#         'name': 'Module name',
#         'credits': 'Credit weight',
#         'level': 'SCQF level'
#         }
#     optionalParams= {
#         'startyear':'First academic year',
#         'endyear': 'Last academic year',
#         'school':'School which manages the Programme'}

    
#     def __init__(self, factory, **kwargs):
#         self.factory=factory
#         self.name = kwargs.get("name")
#         self.definition = kwargs.get("definition")
#         self.id = kwargs.get("id")
#         if self.id is None:
#             self.create()
            
        
#     def create(self):
#         '''
#         Creates a new database entry for the TeachingActivityType entity

#         Returns
#         -------
#         None.

#         '''
#         insertsql = "Insert into TeachingActivityType (name, definition) VALUES (%s,%s)"
#         cursor = self.factory.db.cursor()
#         cursor.excecute(insertsql, (self.name, self.definition))
#         self.id = cursor.lastrowid



class TeachingActivity (Node):
    ntype='TeachingActivity'
    requiredParams = {
        'year': 'AcademicYear',
        'type': 'Activity type'
        }
    optionalParams= {
        'duration':'length of activity'
        }

    def __init__(self, factory, **kwargs):
        #description, duration, TAtype, version,previous_TA, moduleID, sequence
        super().__init__(factory, **kwargs)
        self.loadILO()
        
    def loadILO(self):
        '''
        Loads ILO links from the database. Does not create new links or ILOs.

        Returns
        -------
        None.

        '''
        self.ILO = {}
        cypher = "MATCH (t:TeachingActivity ) -[a]-(b:ActivityILO) WHERE elementID(t) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation, target = t.items()[0:2]
            self.ILO[target.element_id]= (dict(target),dict(relation))
    def map_ilo(self, ilo):
        '''
        Associates or updates a Module ILO mapping to a Module.
    
        Parameters
        ----------
        ilo : ModuleILO 
            Module ILO object
        year : text describing the academic year
            Must be either the start year, or for removing an ILO the last year for which it will be relevant.
        remove : Boolean, optional
            Flag to say whether the ILO should be dissociated from the Module. The default is False.
    
        Returns
        -------
        None.
    
        '''
        
        if ilo.element_id in self.ILO:
            return
        cypher = "MERGE (t:TeachingActivity) -[b:HAS_ILO ]->(i:ActivityILO) where elementID(t)=$tid AND elementID(i) = $iid RETURN b"
        records,_,_ = self.factory.db.execute_query(cypher,tid=self.element_id, iid=ilo.element_id, database_=self.factory.dbname)
        if records:
            relation=records[0].items()[0][1]
            self.ILO[ilo.element_id]=(dict(ilo),dict(relation))
            

    def assign_to_module(self, module):
        '''
        Add an activity to a module

        Parameters
        ----------
        module : A TModule object to attach the activity to.
            DESCRIPTION.

        Returns
        -------
        None.

        '''
        module.map_activity(self)
        self.module=module
        

        
        
        
class ILO(Node):
    ntype='ILO'
    ILO_KNOWLEDGE = 1
    ILO_UNDERSTANDING=2
    ILO_SKILL=3
    ILO_ATTITUDE=4
    requiredParams = {
        }
    optionalParams= {
        'SOLO': 'Solo level of the ILO'
        }
    def __init__(self, factory,**kwparams):
        super().__init__(factory,**kwparams)
    
    

class ActivityILO(ILO):

    bloomterms = ('None','Remember',	'Understand',	'Apply',	'Analyze',	'Evaluate',	'Create')
    bloommap ={
        }
    categories = ('Knowledge','Understanding', 'Skill', 'Attitude')

    def __init__(self, factory, **kwargs):
        super().__init__(factory, **kwargs)
        self.milo=None
        self._get_milo()
      
    def _get_milo(self):
        '''
        Retrieves and assigns any mapped MILO to this AILO. 
        Raises an exception if more than one MILO is mapped in the DB

        Returns
        -------
        relation object between AILO and AILO

        '''
       
        cypher = "MATCH (a:ActivityILO) -[b:MAPS_TO] - (m:ModuleILO) WHERE elementID(a)=$id RETURN a,b,m"
        records,_,_ = self.factory.db.execute_query(cypher,id=self.element_id, 
                                                    database_=self.factory.dbname)
        if records:
            if len(records)>1:
                raise Exception('Maps to multiple Module ILOs')
            
            target=records[0].items()[2][1]
            self.milo = target    
    
    def getActivitiesForILO(self):
        '''
        Retrieve all Activities with this ILO

        Returns
        -------
        List of TeachingActivity

        '''
        activities = self.factory.getElementsForElement(self,'TeachingActivity',max_steps=1)
        return activities

    def supports(self, milo=None):
        '''
        Maps (if milo given) and returns the moduleILO, if present,
        to which this activity ILO maps
        
        
        Parameters
        ----------
        milo : ModuleILO, optional
            ModuleILO object which should be mapped to this activity ILO. 
            The default is None.
        
        Returns
        -------
        ModuleILO if assigned, None if not assigned
        
        '''
        if self.milo and milo:
            if milo != self.milo:
                #delete existing relation and replace.
                cypherd = "MATCH (a:ActivityILO) -[b:MAPS_TO] - (m:ModuleILO) WHERE elementID(a)=$id DELETE b"
                records,_,_ = self.factory.db.execute_query(cypherd,id=self.element_id, 
                                                            database_=self.factory.dbname)
            else:
                return self.milo
        if milo:
            cyphera = "MERGE (a:ActivityILO) -[MAPS_TO]-> (m:ModuleILO) WHERE elementID(a)=$id AND elementID(m) =$mid"
            records,_,_ = self.factory.db.execute_query(cyphera,
                                                        id=self.element_id, mid=milo.element_id,
                                                        database_=self.factory.dbname)
            self.milo=milo
        return self.milo

    
class ModuleILO(ILO):
     
    def __init__(self, factory, **kwargs):
        super().__init__(factory, **kwargs)
        self.pilo=None
        self._get_pilo()

    def _get_pilo(self):
        '''
        Retrieves and assigns any mapped PILO to this MILO. 
        Raises an exception if more than one PILO is mapped in the DB

        Returns
        -------
        relation object between MILO and PILO

        '''
        
        cypher = "MATCH (m:ModuleILO) -[a:MAPS_TO] - (p:ProgrammeILO) WHERE elementID(m)=$id RETURN m,a,p"
        records,_,_ = self.factory.db.execute_query(cypher,id=self.element_id, 
                                                    database_=self.factory.dbname)
        if records:
            if len(records)>1:
                raise Exception('Maps to multiple Programme ILOs')
            
            target=records[0].items()[2][1]
            self.pilo = target

    def getModulesForILO(self):
        '''
        Retrieve all Activities with this ILO
         
        Returns
        -------
        List of Modules
         
        '''
        modules = self.factory.getElementsForElement(self,'Module',max_steps=1)
        return modules
            
    def supports(self, pilo=None):
        '''
        Maps (if pilo given) and returns the progammeILO, if present to which this module ILO maps
        
        
        Parameters
        ----------
        pilo : ProgrammeILO, optional
            ProgrammeILO object which should be mapped to this module ILO. 
            The default is None.
        
        Returns
        -------
        ProgrammeILO if assigned, None if not assigned
        
        '''
        if self.pilo and pilo:
            if pilo != self.pilo:
                #delete existing relation and replace.
                cypherd = "MATCH (m:ModuleILO) -[a:MAPS_TO] - (p:ProgrammeILO) WHERE elementID(m)=$id DELETE a"
                records,_,_ = self.factory.db.execute_query(cypherd,id=self.element_id, 
                                                            database_=self.factory.dbname)
            else:
                return self.pilo
        if pilo:
            cyphera = "MERGE (m:ModuleILO) -[MAPS_TO]-> (p:ProgrammeILO) WHERE elementID(m)=$id AND elementID(p) =$pid"
            records,_,_ = self.factory.db.execute_query(cyphera,
                                                        id=self.element_id, pid=pilo.element_id,
                                                        database_=self.factory.dbname)
            self.pilo=pilo
        return self.pilo

    
class ProgrammeILO(ILO):
   
     
    def __init__(self, factory, **kwargs):
        super().__init__(factory, **kwargs)
        self.pilo=None

    def getProgrammesForILO(self):
        '''
        Retrieve all Activities with this ILO
         
        Returns
        -------
        List of Modules
         
        '''
        programmes = self.factory.getElementsForElement(self,'Programme',max_steps=1)
        return programmes
            
     
    
class AssessmentType():
    '''
    CREATE or REPLACE TABLE AssessmentType (
    ID integer not null primary key auto_increment,
    name text not null,
    definition text not null
    );
    '''

    def __init__(self, factory, **kwargs):
        self.factory = factory
        self.name = kwargs.get("name")
        self.definition = kwargs.get("definition")
        self.id = kwargs.get("id")
        if self.id is None:
            self.create()
            
        
    def create(self):
        '''
        Creates a new database entry for the AssessmentType entity

        Returns
        -------
        None.

        '''
        insertsql = "Insert into AssessmentType (name, definition) VALUES (%s,%s)"
        cursor = self.factory.db.cursor()
        cursor.excecute(insertsql, (self.name,self.definition))
        self.id = cursor.lastrowid   

class Assessment(TeachingActivity):
    ntype='Assessment'
    requiredParams = {
        'year': 'AcademicYear',
        'type': 'Activity type',
        'weighting': 'percentage weighting',
        'code': 'SEQ number'
        }
    optionalParams= {
        }

    def __init__(self, factory,**kwargs):
        super().__init__(factory, **kwargs)
        



In [129]:
factory = CurriculumFactory(driver, 'curriculumdb')

In [130]:
bs32031=factory.get_or_create_Element('Module', **newmodparams)

In [131]:
bs32031

Module: [code: BS32031, name: Bioinformatics Research Skills 1, credits: 15, scqflevel: 3, shelevel: 9, semester: 2] ID: 4:4f947b4e-c5e2-45c5-b439-76aa1c96d891:0

In [48]:
[x.__name__ for x in globals()['Module'].__mro__]

['Module', 'Node', 'object']

In [ ]:
doc = docx.Document("BS32008 Plant Science Specification QASv3.docx")

In [ ]:
for p in doc.paragraphs:
    print(p.text)


In [ ]:
doc.tables[0].row_cells(0)[1].text



In [ ]:
tcount=0
structure ={}
for t in doc.tables:
    print(f'Table {tcount} ({len(t.rows)} rows)')
    tcount+=1
    for r in range(len(t.rows)):
        print(f'row {r} [{t.row_cells(r)[0].text}] [{t.row_cells(r)[1].text}])')
        structure[t.row_cells(r)[0].text.split()[0]] =t.row_cells(r)[1].text
        

In [ ]:
structure.keys()